In [1]:
!pip install accelerate -U
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.3 MB/s eta 0:00:00


In [1]:
!pip install evaluate
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.5 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

dataset = load_dataset('yangwang825/reuters-21578')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/yangwang825___json/yangwang825--reuters-21578-7a62a981513ce198/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 5485
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2189
    })
})

In [6]:
dataset['train'][10]

{'text': 'computer language research in clri th qtr shr loss cts vs loss cts net loss vs loss revs mln vs mln qtly div three cts vs three cts prior year shr profit two cts vs profit cts net profit vs profit revs mln vs mln note dividend payable april one to shareholders of record march reuter',
 'label': 2}

In [3]:
labels = set([i['label'] for i in dataset['train']])

In [4]:
labels

{0, 1, 2, 3, 4, 5, 6, 7}

In [5]:
from transformers import AutoTokenizer


tokenizer_bert = AutoTokenizer.from_pretrained('bert-base-uncased')

In [6]:
def tokenize_function_bert(examples):
    return tokenizer_bert(examples["text"], padding="max_length", truncation=True)


tokenized_datasets_bert = dataset.map(tokenize_function_bert, batched=True)

Map:   0%|          | 0/5485 [00:00<?, ? examples/s]

Map:   0%|          | 0/2189 [00:00<?, ? examples/s]

In [7]:
small_train_dataset_bert = tokenized_datasets_bert['train'].shuffle(seed=42).select(range(1000))
small_eval_dataset_bert = tokenized_datasets_bert['test'].shuffle(seed=42).select(range(1000))

In [8]:
from transformers import AutoModelForSequenceClassification


model_bert = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=8)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [9]:
from transformers import TrainingArguments, Trainer

training_args_bert = TrainingArguments(output_dir="test_trainer_bert", evaluation_strategy="epoch", num_train_epochs=4,
                                      per_device_train_batch_size=8, per_device_eval_batch_size=8)

In [10]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [11]:
trainer_bert = Trainer(
    model=model_bert,
    args=training_args_bert,
    train_dataset=small_train_dataset_bert,
    eval_dataset=small_eval_dataset_bert,
    compute_metrics=compute_metrics,
)

In [12]:
trainer_bert.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.279036,0.925000
2,No log,0.135686,0.966000
3,No log,0.131494,0.969000
4,0.215400,0.114013,0.975000


TrainOutput(global_step=500, training_loss=0.21543785095214843, metrics={'train_runtime': 535.527, 'train_samples_per_second': 7.469, 'train_steps_per_second': 0.934, 'total_flos': 1052500918272000.0, 'train_loss': 0.21543785095214843, 'epoch': 4.0})

In [13]:
trainer_bert.evaluate()

{'eval_loss': 0.11401291936635971,
 'eval_accuracy': 0.975,
 'eval_runtime': 36.7466,
 'eval_samples_per_second': 27.213,
 'eval_steps_per_second': 3.402,
 'epoch': 4.0}